In [1]:
import numpy as np
import cv2
import pandas as pd
import imageio as im
from PIL import Image
import os
import dlib
from imutils.face_utils.helpers import rect_to_bb

In [2]:
pwd

'C:\\Users\\Bharadwaj\\IIST\\COMPUTER VISION\\Project'

In [5]:
cd M:\IIST\SEM 2\COMPUTER VISION\Project\FaceAttributeRecognition\glasses_dataset

M:\IIST\SEM 2\COMPUTER VISION\Project\FaceAttributeRecognition\glasses_dataset


In [6]:
from tqdm import tqdm

In [7]:
ls

 Volume in drive M is MACHINE LEARNING
 Volume Serial Number is 4E23-4B98

 Directory of M:\IIST\SEM 2\COMPUTER VISION\Project\FaceAttributeRecognition\glasses_dataset

13-02-2021  11:38    <DIR>          .
13-02-2021  11:38    <DIR>          ..
13-02-2021  11:38    <DIR>          glass
13-02-2021  11:38    <DIR>          w_glass
               0 File(s)              0 bytes
               4 Dir(s)  259,269,644,288 bytes free


In [8]:
## Cropping the faces from images

# detector = dlib.get_frontal_face_detector()
# dest = 'glass/'
# img_dir = ['glasses/']
# for im_dir in img_dir:
#     images = os.listdir(im_dir)
#     for i,image in enumerate(images):
#         img = cv2.imread(im_dir + image)
#         gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         rects = detector(gray, 1) 
#         for j,rect in enumerate(rects):
#             (x, y, w, h) = rect_to_bb(rect)
# #             cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,255),1)
#             sub_face = gray[y:y+h, x:x+w]
#             cv2.imwrite(dest+str(i)+'_'+str(j)+'.jpg',sub_face)

## Model 1 : Reading the Dataset and Resizing each image to 36 * 36

In [9]:
all_images = []
labels = []
img_dir = ['glass/','w_glass/']
for im_dir in tqdm(img_dir):
    images = os.listdir(im_dir)
    for image in images:
        img = cv2.imread(im_dir + image)
        img = cv2.resize(img, (36, 36)) # need to resize the image into common size
        all_images.append(img)
        labels.append(img_dir.index(im_dir))

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 12.21it/s]


In [10]:
type(all_images)

list

In [11]:
img = cv2.imread('glass/101_0.jpg')
img.shape

(75, 75, 3)

In [12]:
all_images[0].shape

(36, 36, 3)

In [13]:
data = np.array(all_images)
data = data.astype('float32')
data /= 255.0
len(data)

592

In [14]:
# np.save(open('glass.npy', 'wb'), data)
# np.save(open('labels.npy', 'wb'), y_train)

In [15]:
from sklearn.preprocessing import OneHotEncoder
labels = np.asarray(labels)
labels = labels.reshape(-1, 1)
ohe_y=OneHotEncoder()
Y=ohe_y.fit_transform(labels)
Y.shape

(592, 2)

In [16]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
x,y = shuffle(data,Y, random_state=42)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

## Creating a Model for an input image Size of 36 * 36

In [1]:
import keras
from keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Model, Sequential
from keras.optimizers import Adam

classifier=Sequential()

classifier.add(Convolution2D(32,3,3,activation='relu',input_shape=(36, 36, 3)))

classifier.add(Convolution2D(64,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))


classifier.add(Convolution2D(128,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Convolution2D(256,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))


classifier.add(Flatten())

classifier.add(Dense(256,activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(2,activation='softmax'))

classifier.compile(Adam(lr=0.0001, decay=1e-6),loss='categorical_crossentropy',metrics=['accuracy'])

Using TensorFlow backend.
C:\Users\Bharadwaj\.conda\envs\gputest\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(36, 36, 3...)`
  
C:\Users\Bharadwaj\.conda\envs\gputest\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Bharadwaj\.conda\envs\gputest\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  from ipykernel import kernelapp as app
C:\Users\Bharadwaj\.conda\envs\gputest\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`


In [18]:
classifier.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=1, shuffle=True, verbose=1 )

Train on 473 samples, validate on 119 samples
Epoch 1/25
473/473 [==============================] - 6s 12ms/step - loss: 0.7000 - accuracy: 0.5074 - val_loss: 0.6867 - val_accuracy: 0.4874
Epoch 2/25
473/473 [==============================] - 5s 11ms/step - loss: 0.5933 - accuracy: 0.6934 - val_loss: 0.4962 - val_accuracy: 0.7815
Epoch 3/25
473/473 [==============================] - 5s 11ms/step - loss: 0.4275 - accuracy: 0.8288 - val_loss: 0.3590 - val_accuracy: 0.8571
Epoch 4/25
473/473 [==============================] - 5s 10ms/step - loss: 0.3396 - accuracy: 0.8478 - val_loss: 0.3531 - val_accuracy: 0.8403
Epoch 5/25
473/473 [==============================] - 5s 10ms/step - loss: 0.3219 - accuracy: 0.8584 - val_loss: 0.3168 - val_accuracy: 0.8655
Epoch 6/25
473/473 [==============================] - 5s 10ms/step - loss: 0.2560 - accuracy: 0.8985 - val_loss: 0.2808 - val_accuracy: 0.8824
Epoch 7/25
473/473 [==============================] - 5s 11ms/step - loss: 0.2174 - accuracy: 0.

In [19]:
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 34, 34, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 5, 5, 256)        

## Creating a Model for an input image Size of 48 * 48

In [20]:
all_images = []
labels = []
img_dir = ['glass/','w_glass/']
for im_dir in img_dir:
    images = os.listdir(im_dir)
    for image in images:
        img = cv2.imread(im_dir + image)
        img = cv2.resize(img, (48, 48)) # need to resize the image into common size
        all_images.append(img)
        labels.append(img_dir.index(im_dir))

In [21]:
all_images[0].shape

(48, 48, 3)

In [22]:
data = np.array(all_images)
data = data.astype('float32')
data /= 255.0
len(data)

592

In [23]:
# np.save(open('glass.npy', 'wb'), data)
# np.save(open('labels.npy', 'wb'), y_train)

from sklearn.preprocessing import OneHotEncoder
labels = np.asarray(labels)
labels = labels.reshape(-1, 1)
ohe_y=OneHotEncoder()
Y=ohe_y.fit_transform(labels)
Y.shape

(592, 2)

In [24]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
x,y = shuffle(data,Y, random_state=42)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)

import keras
from keras.layers import Dense, Input, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Model, Sequential
from keras.optimizers import Adam

classifier=Sequential()

classifier.add(Convolution2D(32,3,3,activation='relu',input_shape=(48, 48, 3)))

classifier.add(Convolution2D(64,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))


classifier.add(Convolution2D(128,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

classifier.add(Convolution2D(256,3,3,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Dropout(0.25))


classifier.add(Flatten())

classifier.add(Dense(256,activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(2,activation='softmax'))

classifier.compile(Adam(lr=0.0001, decay=1e-6),loss='categorical_crossentropy',metrics=['accuracy'])


classifier.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=25, batch_size=1, shuffle=True, verbose=1 )

C:\Users\Bharadwaj\.conda\envs\gputest\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(48, 48, 3...)`
  
C:\Users\Bharadwaj\.conda\envs\gputest\lib\site-packages\ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  app.launch_new_instance()
C:\Users\Bharadwaj\.conda\envs\gputest\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
C:\Users\Bharadwaj\.conda\envs\gputest\lib\site-packages\ipykernel_launcher.py:24: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`


Train on 473 samples, validate on 119 samples
Epoch 1/25
473/473 [==============================] - 12s 25ms/step - loss: 0.6983 - accuracy: 0.5285 - val_loss: 0.6795 - val_accuracy: 0.5126
Epoch 2/25
473/473 [==============================] - 12s 25ms/step - loss: 0.5451 - accuracy: 0.7188 - val_loss: 0.4108 - val_accuracy: 0.8235
Epoch 3/25
473/473 [==============================] - 12s 25ms/step - loss: 0.3352 - accuracy: 0.8562 - val_loss: 0.3516 - val_accuracy: 0.8487
Epoch 4/25
473/473 [==============================] - 12s 26ms/step - loss: 0.2712 - accuracy: 0.8964 - val_loss: 0.2914 - val_accuracy: 0.8824
Epoch 5/25
473/473 [==============================] - 12s 25ms/step - loss: 0.2364 - accuracy: 0.9091 - val_loss: 0.3240 - val_accuracy: 0.8655
Epoch 6/25
473/473 [==============================] - 13s 27ms/step - loss: 0.2307 - accuracy: 0.9154 - val_loss: 0.2992 - val_accuracy: 0.8739
Epoch 7/25
473/473 [==============================] - 12s 26ms/step - loss: 0.1881 - accur

In [75]:
classifier.save('glass_model.h5')

In [18]:
# import keras
# target_size = 224
# from keras.applications.mobilenetv2 import MobileNetV2
# from keras.layers import Dense, Input, Dropout
# from keras.models import Model

# def build_model( ):
#     input_tensor = Input(shape=(target_size, target_size, 3))
#     base_model = MobileNetV2(include_top=False, weights='imagenet', input_tensor=input_tensor,
#         input_shape=(target_size, target_size, 3), pooling='avg')

#     for layer in base_model.layers:
#         layer.trainable = True  # trainable has to be false in order to freeze the layers
#     op = Dense(1, activation = 'softmax')(base_model.output)

#     output_tensor = Dense(1, activation='softmax')(op)

#     model = Model(inputs=input_tensor, outputs=output_tensor)

#     return model

# from keras.optimizers import Adam
# model = build_model()
# model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# model.fit(X_train, y_train, epochs = 5, verbose = 1,batch_size=1 ,validation_data=(X_test,y_test))